@author: Manal Rahal - 21 Nov 2022 <br>
@title: Models testing on the big dataset, predicting peak width

In [ ]:
! pip install seqfold

In [ ]:
import os
mydir = os.getcwd() # would be the MAIN folder
mydir

In [ ]:
! python Encode_funcs_test.py

In [ ]:
! python supporting_funcs.py

In [ ]:
import numpy as np
import pandas as pd
import os
import time
from datetime import datetime
from itertools import product, combinations
import sklearn.model_selection as skl_ms
import sklearn.metrics as skl_me
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
import sklearn.svm as svm
import matplotlib.pyplot as plt 
from sklearn import preprocessing
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, make_scorer, r2_score
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn import preprocessing
import pickle
from sklearn.utils import shuffle

In [ ]:
import Encode_funcs_test
from Encode_funcs_test import *
from Encode_funcs_test import encode_input

In [ ]:
import supporting_funcs
from supporting_funcs import get_null_percentage, isnullcolumns, isinfinitecolumns, find_outliers_IQR, generate_ID
from supporting_funcs import draw_scatter, is_lost_sulfur, count_sulfur, return_phosphated_seq, is_phosphated, percentage_phosphated

## **Loading and Encoding Retention Time Dataset**

In [ ]:
#__Loading data from excel file, each gradient is in a sheet

xls = pd.ExcelFile('../data/oligodata121022.xlsx')

In [ ]:
oligo_G11 = pd.read_excel(xls, sheet_name='11min gradient')
oligo_G22 =pd.read_excel(xls, sheet_name='22min gradient')
oligo_G33 =pd.read_excel(xls, sheet_name='44min gradient')

In [ ]:
oligo_G11.info()

In [ ]:
oligo_G22.info()

In [ ]:
oligo_G33.info()

In [ ]:
#check all sequences columns in G1, G2 dataset match before merging

#merge_data = pd.DataFrame()
#merge_data['S1'] = oligo_G11['Sequence']
#merge_data['S2'] = oligo_G22['Sequence']
#merge_data['all_matching'] = merge_data.apply(lambda x: x.S1 == x.S2, axis = 1)
#merge_data[ merge_data['all_matching'] == False]

#therefore S1 and S2 columns match, we can merge width1 and width 2.
#--------------------------------------------------------------------

In [ ]:
#check all sequences columns in G1, G2, G3 dataset match before merging
#merge_data = pd.DataFrame()
#merge_data['S1'] = oligo_G11['Sequence']
#merge_data['S2'] = oligo_G22['Sequence']
#merge_data['S3'] = oligo_G33['Sequence']
#merge_data['all_matching'] = merge_data.apply(lambda x: x.S1 == x.S2 == x.S3, axis = 1)
#merge_data[ merge_data['all_matching'] == False]

#therefore S1 and S3 dont match, we need to setup a condition for matching before merging G3 data into merge_data.
#-----------------------------------------------------------------------------------------------------------------

In [ ]:
oligo_G11['gradient'] = 'G1'
oligo_G22['gradient'] = 'G2'
oligo_G33['gradient'] = 'G3'
oligo_G33.head()

In [ ]:
frames = [oligo_G11, oligo_G22, oligo_G33]
merge_data = pd.concat(frames, axis = 0)
merge_data

In [ ]:
merge_data.info()

In [ ]:
print (isnullcolumns(merge_data))

In [ ]:
# drop rows that has a null value in the width column

merge_data = merge_data.dropna(axis=0, how='any', subset=['width'])

In [ ]:
print (isnullcolumns(merge_data))

In [ ]:
merge_data.head()

In [ ]:
merge_data = merge_data.reset_index()

In [ ]:
merge_data = merge_data.astype({'Sequence':'string'})

In [ ]:
merge_data.info()

In [ ]:
non_Phosp_data = merge_data[ merge_data["Sequence"].str.contains("P=O") == False]

In [ ]:
non_Phosp_data[non_Phosp_data['gradient'] == 'G2'].shape

In [ ]:
non_Phosp_data.info()

In [ ]:
non_Phosp_data.shape

In [ ]:
non_Phosp_data

In [ ]:
target_data = non_Phosp_data.loc[:,['width']]
print(target_data.shape)
target_data.head()

In [ ]:
input_data = non_Phosp_data.loc[:, ~non_Phosp_data.columns.isin(['width'])]
print(input_data.shape)
input_data.head()

In [ ]:
#input_data.to_excel("PW_input_data.xlsx")

# Machine Learning 

In [ ]:
#### RUN ONLY ONCE THEN READ THE DATASETS FROM THE EXPERTED FILES
#train_ratio = 0.7
#test_ratio = 0.3

#xtrain, xtest, ytrain, ytest = train_test_split(input_data, target_data, test_size=test_ratio, random_state=42, shuffle =True)

#print('xtrain ', xtrain.shape,'\n')
#print('ytrain ', ytrain.shape,'\n')
#print('xtest ', xtest.shape,'\n')
#print('ytest ', ytest.shape,'\n')

In [ ]:
#### RUN ONLY ONCE THEN READ THE DATASETS FROM THE EXPORTED FILES
#----------------------------------------------------------------#

#xtrain.to_excel("PW_xtrain.xlsx")
#ytrain.to_excel("PW_ytrain.xlsx")
#xtest.to_excel("PW_xtest.xlsx")
#ytest.to_excel("PW_ytest.xlsx")

In [ ]:
x_train = pd.read_excel('PW_xtrain.xlsx')
y_train = pd.read_excel('PW_ytrain.xlsx')

In [ ]:
x_test = pd.read_excel('PW_xtest.xlsx')
y_test = pd.read_excel('PW_ytest.xlsx')

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
x_test.shape

In [ ]:
y_test.shape

In [ ]:
x_train.head()

In [ ]:
y_train.head()

In [ ]:
x_train = x_train.loc[:,['Sequence','tR1','Vinj','gradient']]
x_train.info()

## Gradient is encoded as category 1,2, 3

In [ ]:
x_train.info()

In [ ]:
x_train["gradient"] = x_train["gradient"].astype('category')
x_train.dtypes

In [ ]:
x_train["gradient_codes"] = x_train["gradient"].cat.codes
x_train.head()

In [ ]:
x_train.head()

In [ ]:
x_train.info()

In [ ]:
df_sequence_input = x_train["Sequence"]
df_sequence_input.shape

In [ ]:
sequence =[]
for i,seq in enumerate (df_sequence_input):
    sequence=np.append(sequence, seq)

In [ ]:
pw_vec = y_train['width']
pw_vec.shape

In [ ]:
features = {}

features['count'] = ['Fa', 'Ft', 'Fg', 'Fc', 'Length']
features ['sulfurcount'] = ['Fsulf']
#features['contact'] = prod('ATGC', repeat = 2)
#features['scontact'] = [f"{comb[0]}{comb[-1]}_{comb[-1]}{comb[0]}" for comb in pair_comb('ATGC', 2)]
features['position'] = ['first position', 'last position']

featvec = ['count','sulfurcount','position']
featcombinations = []

for i in range(len(featvec)):
    for c in list(combinations(featvec, i + 1)):
        featcombinations.append(list(c))

featvec

In [ ]:
inputvec_train = encode_input(sequence, featvec)
inputvec_train.head()

In [ ]:
inputvec_train["tR1"] = x_train["tR1"]
inputvec_train["Vinj"] = x_train["Vinj"]
inputvec_train["gradient_codes"] = x_train["gradient_codes"]

In [ ]:
inputvec_train.info()

In [ ]:
inputvec_train["First"] = inputvec_train["First"].astype('category')
inputvec_train["Last"] = inputvec_train["Last"].astype('category')
inputvec_train["First_codes"] = inputvec_train["First"].cat.codes
inputvec_train["Last_codes"] = inputvec_train["Last"].cat.codes
inputvec_train.info()

In [ ]:
cols = inputvec_train.select_dtypes(include=['object'])

for col in  cols:
    inputvec_train[col] = pd.to_numeric(inputvec_train[col], errors='coerce')

inputvec_train.info()

In [ ]:
inputvec_train = inputvec_train.drop(['First', 'Last'], axis=1)
inputvec_train.info()

In [ ]:
print(isnullcolumns(inputvec_train))

In [ ]:
x_test.info()

In [ ]:
#Apply same transformations for the test set
#"""""""""""""""""""""""""""""""""""""""""""

xtest = x_test.loc[:,['Sequence','tR1','Vinj','gradient']]
xtest["gradient"] = xtest["gradient"].astype('category')
xtest["gradient_codes"] = xtest["gradient"].cat.codes

df_sequence_test = xtest["Sequence"]
df_sequence_test.shape
sequence_ts =[]
for i,seq in enumerate (df_sequence_test):
    sequence_ts=np.append(sequence_ts, seq)
#print(len(sequence_ts))

ytest = y_test['width']
#print (ytest.shape)

inputvec_test = encode_input(sequence_ts, featvec)
inputvec_test.head()
inputvec_test["tR1"] = xtest["tR1"]
inputvec_test["Vinj"] = xtest["Vinj"]
inputvec_test["gradient_codes"] = xtest["gradient_codes"]

#inputvec_test.info()

inputvec_test["First"] = inputvec_test["First"].astype('category')
inputvec_test["Last"] = inputvec_test["Last"].astype('category')
inputvec_test["First_codes"] = inputvec_test["First"].cat.codes
inputvec_test["Last_codes"] = inputvec_test["Last"].cat.codes

cols = inputvec_test.select_dtypes(include=['object'])

for col in  cols:
    inputvec_test[col] = pd.to_numeric(inputvec_test[col], errors='coerce')

inputvec_test = inputvec_test.drop(['First', 'Last'], axis=1)

print(inputvec_test.info())
print(isnullcolumns(inputvec_test))

In [ ]:
params = {
            'max_depth': [2,4,6,8,10,20,50,100],
            'learning_rate': [0.001, 0.01, 0.1, 1],
            'n_estimators': [100, 200, 500, 1000],
            'max_leaf_nodes': [2, 5, 10,20]
        }

In [ ]:
start = time.time()

model = GradientBoostingRegressor()
            
model.fit(inputvec_train, pw_vec)

print(f'\n --- %s seconds --- % {time.time() - start}')

In [ ]:
acc_train = round(model.score(inputvec_train, pw_vec), 3)
print("training r2", acc_train)

In [ ]:
predtr = model.predict(inputvec_train)

rmsetr = skl_me.mean_squared_error(pw_vec, predtr, squared = False)

print (rmsetr)

In [ ]:
ytest

In [ ]:
predts = model.predict(inputvec_test)

rmse_ts = skl_me.mean_squared_error(ytest, predts, squared = False)

r2_ts = r2_score(ytest, predts)

print (f'Best Features GB hyperparameters - testing scores')
print (f'---------------')
print(f"R2: %.3f" % r2_ts)
print (f"RMSE: %.3f" % rmse_ts)

In [ ]:
print("the not hypertuned model", model)

In [ ]:
df_predicted = pd.DataFrame(data=predts)

df_predicted = df_predicted.set_index(ytest.index)

df_predicted.rename(columns={0: "predicted"}, inplace=True)
df_error = pd.concat([ytest, df_predicted], axis=1)
df_error.columns = ["Y_observed", "Y_predicted"]
df_error.shape

In [ ]:
df_error["residuals"] = df_error["Y_observed"] - df_error["Y_predicted"]
df_error.head()

In [ ]:
plt.figure(figsize=(12, 7))
plt.hist(df_error["residuals"], bins=15, color="teal", edgecolor="grey", linewidth=1.5, alpha=0.6)
plt.axvline(0, color="white", linestyle="dashed", linewidth=2)
plt.show()

In [ ]:
plt.figure(figsize=(10,8))
#plt.scatter(test_tRvec, pred, c='#6495ED', s=14)

#p1 = max(max(pred), max(test_tRvec))
#p2 = min(min(pred), min(test_tRvec))
plt.plot([0, 0.7], [0, 0.7], 'k-')

plt.scatter(pw_vec, predtr, color = 'magenta')
plt.scatter(ytest, predts, color = 'green', alpha=0.4)
plt.xlabel("Observed peak width", fontsize=14)
plt.ylabel("Predicted peak width", fontsize=14)
plt.title("\nThe scatter plot shows that GB is significantly underestimating peak width \n especially with the G3 datapoints as seen in the error analysis")
plt.show()
    
#plt.plot([p1, p2], [p1, p2], 'k-')
#plt.axis('equal')
#plt.savefig('PW_G2_GB.pdf', format='pdf', dpi=1200, bbox_inches='tight')
#plt.show()

In [ ]:
from matplotlib import pyplot

plt.figure(figsize=(8, 6))

bins = np.linspace(0, 0.8, 100)

pyplot.hist(pw_vec, bins, alpha=0.6, color="blue", edgecolor="white")
plt.savefig('PWDistribution.pdf', format='pdf', dpi=1200, bbox_inches='tight')
pyplot.show()

In [ ]:
pd.options.display.max_rows = 500
df_error.sort_values('residuals').head(20)

In [ ]:
xtest.iloc[[158, 293, 52,327,96,451,335,414,120,120,97,315,147,421,134,277,371,464,196,31,129]]

In [ ]:
xtest

## Gradient is Float 11., 22. , 44.

In [ ]:
x_train['gradient'].replace(to_replace = 'G1', value =11., inplace=True)
x_train['gradient'].replace(to_replace = 'G2', value =22., inplace=True)
x_train['gradient'].replace(to_replace = 'G3', value =44., inplace=True)

In [ ]:
x_train.head()

In [ ]:
x_train.info()

In [ ]:
df_sequence_input = x_train["Sequence"]

df_sequence_input.shape

sequence =[]
for i,seq in enumerate (df_sequence_input):
    sequence=np.append(sequence, seq)
    
sequence.shape

In [ ]:
pw_vec = y_train['width']
pw_vec.shape

In [ ]:
features = {}

features['count'] = ['Fa', 'Ft', 'Fg', 'Fc', 'Length']
features ['sulfurcount'] = ['Fsulf']
#features['contact'] = prod('ATGC', repeat = 2)
#features['scontact'] = [f"{comb[0]}{comb[-1]}_{comb[-1]}{comb[0]}" for comb in pair_comb('ATGC', 2)]
features['position'] = ['first position', 'last position']

featvec = ['count','sulfurcount','position']
featcombinations = []

for i in range(len(featvec)):
    for c in list(combinations(featvec, i + 1)):
        featcombinations.append(list(c))

featvec

In [ ]:
input_train = encode_input(sequence, featvec)
input_train.head()

In [ ]:
input_train["tR1"] = x_train["tR1"]
input_train["Vinj"] = x_train["Vinj"]
input_train["gradient"] = x_train["gradient"]
input_train.head(10)

In [ ]:
input_train.info()

In [ ]:
input_train["First"] = input_train["First"].astype('category')
input_train["Last"] = input_train["Last"].astype('category')
input_train["First_codes"] = input_train["First"].cat.codes
input_train["Last_codes"] = input_train["Last"].cat.codes
input_train.info()

In [ ]:
input_train['gradient'] = input_train["gradient"].astype('object')

In [ ]:
cols = input_train.select_dtypes(include=['object'])

for col in  cols:
    print (col)
    input_train[col] = pd.to_numeric(input_train[col], errors='coerce')

input_train.info()

In [ ]:
input_train = input_train.drop(['First', 'Last'], axis=1)
input_train.info()

In [ ]:
print(isnullcolumns(input_train))

In [ ]:
input_train.info()

In [ ]:
xtest.info()

In [ ]:
#Apply same transformations for the test set
#"""""""""""""""""""""""""""""""""""""""""""

xtest = x_test.loc[:,['Sequence','tR1','Vinj','gradient']]
xtest['gradient'].replace(to_replace = 'G1', value =11., inplace=True)
xtest['gradient'].replace(to_replace = 'G2', value =22., inplace=True)
xtest['gradient'].replace(to_replace = 'G3', value =44., inplace=True)

df_sequence_test = xtest["Sequence"]
df_sequence_test.shape
sequence_ts =[]
for i,seq in enumerate (df_sequence_test):
    sequence_ts=np.append(sequence_ts, seq)
#print(len(sequence_ts))

ytest = y_test['width']
#print (ytest.shape)

input_test = encode_input(sequence_ts, featvec)
input_test.head()
input_test["tR1"] = xtest["tR1"]
input_test["Vinj"] = xtest["Vinj"]
input_test["gradient"] = xtest["gradient"]

#inputvec_test.info()

input_test["First"] = input_test["First"].astype('category')
input_test["Last"] = input_test["Last"].astype('category')
input_test["First_codes"] = input_test["First"].cat.codes
input_test["Last_codes"] = input_test["Last"].cat.codes

cols = input_test.select_dtypes(include=['object'])

for col in  cols:
    input_test[col] = pd.to_numeric(input_test[col], errors='coerce')

input_test = input_test.drop(['First', 'Last'], axis=1)

print(input_test.info())

print(isnullcolumns(input_test))

In [ ]:
start = time.time()

model1 = GradientBoostingRegressor()
            
model1.fit(input_train, pw_vec)

print(f'\n --- %s seconds --- % {time.time() - start}')

In [ ]:
acc_train1 = round(model1.score(input_train, pw_vec), 3)
print("training r2", acc_train1)

In [ ]:
predtr1 = model1.predict(input_train)

rmsetr1 = skl_me.mean_squared_error(pw_vec, predtr1, squared = False)

print (rmsetr1)

In [ ]:
predts1 = model1.predict(input_test)

rmse_ts1 = skl_me.mean_squared_error(ytest, predts1, squared = False)

r2_ts1 = r2_score(ytest, predts1)

print (f'Best Features GB hyperparameters - testing scores')
print (f'---------------')
print(f"R2: %.3f" % r2_ts1)
print (f"RMSE: %.3f" % rmse_ts1)

In [ ]:
plt.figure(figsize=(10,8))
#plt.scatter(test_tRvec, pred, c='#6495ED', s=14)

#p1 = max(max(pred), max(test_tRvec))
#p2 = min(min(pred), min(test_tRvec))
#plt.plot([0, 0.7], [0, 0.7], 'k-')

plt.scatter(pw_vec, predtr1, color = 'magenta')
plt.scatter(ytest, predts1, color = 'green', alpha=0.4)
plt.xlabel("Observed peak width", fontsize=14)
plt.ylabel("Predicted peak width", fontsize=14)
plt.title("\nThe scatter plot shows that GB is significantly underestimating peak width")
plt.show()
    
#plt.plot([p1, p2], [p1, p2], 'k-')
#plt.axis('equal')
#plt.savefig('PW_G2_GB.pdf', format='pdf', dpi=1200, bbox_inches='tight')

### Testing random search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

randomsearch = RandomizedSearchCV(
    estimator=model3,
    param_distributions=param_grid,
    scoring = ['neg_root_mean_squared_error'], 
    refit = 'neg_root_mean_squared_error',
    cv=5,
    n_jobs=-1,
    verbose=2,
    n_iter=100)

# perform hyperparamter tuning (while timing the process)
time_start = time.time()
randomsearch.fit(input_train, pw_vec)
time_random = time.time() - time_start

# store result in a data frame 
grid_values = [[100, randomsearch.best_index_+1, randomsearch.best_score_, time_random]]
results_random = pd.DataFrame(grid_values, columns = columns)

In [ ]:
summary = results_grid.append(results_random)
summary.index = ['Random Search']
summary

In [ ]:
#Random Search best estimator
best_random = randomsearch.best_estimator_
best_random

In [ ]:
acc_tr_grid = round(best_grid.score(input_train, pw_vec), 3)
print("grid search training r2", acc_tr_grid)

In [ ]:
predtr_grid = best_grid.predict(input_train)

rmse_tr_grid = skl_me.mean_squared_error(pw_vec, predtr_grid, squared = False)
print (f"Grid search rmse: %.3f" % rmse_tr_grid)

In [ ]:
predts_grid = best_grid.predict(input_test)

rmse_ts_grid = skl_me.mean_squared_error(ytest, predts_grid, squared = False)

r2_ts_grid = r2_score(ytest, predts_grid)

print (f'Grid search Optmized GB - testing scores')
print (f'---------------')
print(f"R2: %.3f" % r2_ts_grid)
print (f"RMSE: %.3f" % rmse_ts_grid)

In [ ]:
plt.figure(figsize=(10,8))
plt.scatter(pw_vec, predtr_grid, color = 'magenta')
plt.scatter(ytest, predts_grid, color = 'green', alpha=0.4)
plt.xlabel("Observed peak width", fontsize=14)
plt.ylabel("Predicted peak width", fontsize=14)
plt.title("\n Grid Search ")
plt.show()
    
#plt.plot([p1, p2], [p1, p2], 'k-')
#plt.axis('equal')
#plt.savefig('PW_G2_GB.pdf', format='pdf', dpi=1200, bbox_inches='tight')

## Random Search

In [ ]:
acc_tr_rand = round(best_random.score(input_train, pw_vec), 3)
print("random search training r2", acc_tr_rand)

In [ ]:
predtr_rand = best_random.predict(input_train)

rmse_tr_rand = skl_me.mean_squared_error(pw_vec, predtr_rand, squared = False)
print (f"Grid search rmse: %.3f" % rmse_tr_rand)

In [ ]:
predts_rand = best_random.predict(input_test)

rmse_ts_rand = skl_me.mean_squared_error(ytest, predts_rand, squared = False)

r2_ts_rand = r2_score(ytest, predts_rand)

print (f'Random search Optmized GB - testing scores')
print (f'---------------')
print(f"R2: %.3f" % r2_ts_rand)
print (f"RMSE: %.3f" % rmse_ts_rand)

In [ ]:
best_random

In [ ]:
plt.figure(figsize=(10,8))
#plt.scatter(test_tRvec, pred, c='#6495ED', s=14)

#p1 = max(max(pred), max(test_tRvec))
#p2 = min(min(pred), min(test_tRvec))
#plt.plot([0, 0.7], [0, 0.7], 'k-')

plt.scatter(pw_vec, predtr_rand, color = 'magenta')
plt.scatter(ytest, predts_rand, color = 'green', alpha=0.4)
plt.xlabel("Observed peak width", fontsize=14)
plt.ylabel("Predicted peak width", fontsize=14)
plt.title("\n Random Search ")
plt.show()
    
#plt.plot([p1, p2], [p1, p2], 'k-')
#plt.axis('equal')
#plt.savefig('PW_G2_GB.pdf', format='pdf', dpi=1200, bbox_inches='tight')

## Save results of random search into text file

In [ ]:
x_train.head()

In [ ]:
x_test.head()

In [ ]:
df_train =pd.DataFrame()
df_train['Sequence'] = x_train['Sequence']
df_train['gradient'] = x_train['gradient']
df_train['ytrain'] = np.array(pw_vec)
df_train['tr_predicted'] = np.array(predtr_rand)
df_train.info()

In [ ]:
x_test['gradient'].replace(to_replace = 'G1', value =11., inplace=True)
x_test['gradient'].replace(to_replace = 'G2', value =22., inplace=True)
x_test['gradient'].replace(to_replace = 'G3', value =44., inplace=True)

In [ ]:
df_test =pd.DataFrame()
df_test['Sequence'] = x_test['Sequence']
df_test['gradient'] = x_test['gradient']
df_test['ytest'] = np.array(ytest)
df_test['ts_predicted'] = np.array(predts_rand)
df_test.info()

In [ ]:
path = r'train_data_Seq.txt'
with open(path, 'a') as f:
    df_train = df_train.to_string()
    f.write(df_train)

In [ ]:
path = r'test_data_Seq.txt'
with open(path, 'a') as f:
    df_test = df_test.to_string()
    f.write(df_test)

In [ ]:
df_train =pd.DataFrame()
df_train['ytrain'] = np.array(pw_vec)
df_train['tr_predicted'] = np.array(predtr_rand)
df_test =pd.DataFrame()
df_test['ytest'] = np.array(ytest)
df_test['ts_predicted'] = np.array(predts_rand)

In [ ]:
path = r'train_data.txt'
with open(path, 'a') as f:
    df_train = df_train.to_string()
    f.write(df_train)

In [ ]:
path = r'test_data.txt'
with open(path, 'a') as f:
    df_test = df_test.to_string()
    f.write(df_test)